### To have communication V2X we'll use MQTT (BLACK JETBOT)

In [ ]:
# Install MQTT
#!pip3 install paho-mqtt

# DON'T FORGET ACTIVATE THE MQTT BROKER

In [3]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import paho.mqtt.client as mqtt
import json
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg, Robot
from IPython.display import display

# Configurar el dispositivo CUDA
device = torch.device('cuda')

#############################################
# Load the TRT optimized model
#############################################

from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))
print("Loaded optimized trained weights")

#############################################
# Creating the Pre-Processing Function
#############################################

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

print("Created the Pre-Processing Function")

#############################################
# Initialize Camera
#############################################

camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)
print("Camera displayed")

#############################################
# Initialize JetBot
#############################################

robot = Robot()
print("Created robot instance")

#############################################
# MQTT Configuration
#############################################

MQTT_BROKER = "192.168.21.1"  # IP of the MQTT broker
MQTT_TOPIC = "jetbot/control"

# Variables globales para almacenar valores de control (inicialmente en 0)
speed_gain = 0.0
steering_gain = 0.0
steering_kd = 0.0
steering_bias = 0.0

# Callback cuando se recibe un mensaje MQTT
def on_message(client, userdata, message):
    global speed_gain, steering_gain, steering_kd, steering_bias  # Aseguramos que modificamos las variables globales
    try:
        data = json.loads(message.payload.decode("utf-8"))
        
        # Actualizar los valores de las variables desde MQTT
        if "speed_gain" in data:
            speed_gain = float(data["speed_gain"])
        if "steering_gain" in data:
            steering_gain = float(data["steering_gain"])
        if "steering_kd" in data:
            steering_kd = float(data["steering_kd"])
        if "steering_bias" in data:
            steering_bias = float(data["steering_bias"])
        
        #print(f"Received MQTT Data: {data}")

    except Exception as e:
        print(f"Error processing MQTT message: {e}")

# Configurar cliente MQTT
client = mqtt.Client()
client.on_message = on_message
client.connect(MQTT_BROKER)

# Suscribirse al tópico
client.subscribe(MQTT_TOPIC)
client.loop_start()  # Iniciar loop en segundo plano

print("Listening for MQTT messages...")

#############################################
# Display JetBot Predictions
#############################################

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)
print("Created sliders to see what JetBot is thinking")

#############################################
# Control Loop Function
#############################################

angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last, speed_gain, steering_gain, steering_kd, steering_bias
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain  # Ahora se usa la variable en lugar del slider
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain + (angle - angle_last) * steering_kd
    angle_last = angle
    
    steering_slider.value = pid + steering_bias
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

#############################################
# Attach Neural Network Execution to Camera
#############################################

camera.observe(execute, names='value')
print("JetBot running...")


Loaded optimized trained weights
Created the Pre-Processing Function


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Camera displayed
Created robot instance
Listening for MQTT messages...


FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Created sliders to see what JetBot is thinking
JetBot running...


In [2]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

camera.stop()

print("Jetbot stopped...")

Jetbot stopped...
